In [ ]:
    @staticmethod
    def process_tools(tools):
        """
        Process a list of tools into Gemini API compatible schemas.
        
        Args:
            tools: List of tools that can be:
                - Functions (schema extracted automatically)
                - Tuples of (function, params_model, [response_model])
                
        Returns:
            Tuple of (processed_tools, tool_callables)
            - processed_tools: List of tool schemas
            - tool_callables: Dict mapping tool names to callable functions
        """
        import inspect
        
        processed_tools = []
        tool_callables = {}
        
        if not tools:
            return processed_tools, tool_callables
        
        for tool in tools:
            if isinstance(tool, tuple) and len(tool) >= 2 and callable(tool[0]):
                # Case: (function, params_model, [response_model]) tuple
                function = tool[0]
                params_model = tool[1]
                response_model = tool[2] if len(tool) > 2 else None
                
                # Convert Pydantic models to schema
                func_name = function.__name__
                func_description = inspect.getdoc(function) or ""
                
                params_schema = GeminiUtils.pydantic_to_schema(params_model)
                
                # Create schema
                schema = {
                    "name": func_name,
                    "description": func_description,
                    "parameters": params_schema
                }
                
                # Add response schema if provided
                if response_model:
                    response_schema = GeminiUtils.pydantic_to_schema(response_model)
                    schema["response"] = response_schema
                
                processed_tools.append(schema)
                tool_callables[func_name] = function
                
            elif callable(tool):
                # Case: Plain function
                schema = GeminiUtils.python_to_function(tool)
                processed_tools.append(schema)
                tool_callables[schema["name"]] = tool
                
            else:
                logger.warning(f"Ignoring unsupported tool type: {type(tool)}")
        
        return processed_tools, tool_callables